In [93]:
# Import libraries
import datetime
import plotly.graph_objects as go
import pandas as pd
import statistics

In [94]:
# Function 1: function takes as input the time interval for grouping and returns the grouped dataframe
def group(param):
    #Read csv file into DataFrame
    df = pd.read_csv('prices.csv')

    # Convert the "TS" column data type to datetime64
    df['TS'] = df['TS'].apply(lambda x: x[:-3])
    df['TS'] = df['TS'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))

    # Aggregate trades into candlesticks based on the provided time interval
    df_agg = df.groupby(pd.Grouper(key='TS', label = 'right', freq=param)).first()
    df_agg.rename(columns = {'PRICE':'Open'}, inplace = True )
    df_agg['Close'] = df.groupby(pd.Grouper(key='TS', label = 'right', freq=param)).last()
    df_agg['High'] = df.groupby(pd.Grouper(key='TS', label = 'right', freq=param)).max()
    df_agg['Low'] = df.groupby(pd.Grouper(key='TS', label = 'right', freq=param)).min()
    df_agg['Date'] = df_agg.index
    df_agg = df_agg.reset_index(drop=True)

    # Replacing gaps with previous values
    df_agg = df_agg.fillna(method='ffill')

    return df_agg


# Function 2: The function takes 2 parameters as input: a smoothing window (n) and a row for smoothing (data). The function returns a list with the values of the smoothed row.
def ema(n, data):
    ema_list = []
    mean_n = statistics.mean(data[:n])
    ema_list.extend([mean_n]*n)
    for i in range(n, len(data)):
        ema_list.append(data[i]*(2/(n+1)) + ema_list[-1]*(1-(2/(n+1))))
    return ema_list


In [102]:
result_df = group('30min')
result_df

ema_list = ema(14,result_df['Close'])

In [103]:
# Сandlestick chart
fig = go.Figure(data=[go.Candlestick(x=result_df['Date'],
                open=result_df['Open'],
                high=result_df['High'],
                low=result_df['Low'],
                close=result_df['Close'], name= 'bar'),])
fig.update_layout(xaxis_rangeslider_visible=False,
                  title="Candlesticks",
                  xaxis_title="Date",
                  yaxis_title="Prices")
fig.show()

In [104]:
# Chart of closing prices and EMA line
fig = go.Figure()
fig.add_trace(go.Scatter(x=result_df['Date'], y=result_df['Close'], name= 'Closing prices',line=dict(color="#0000ff")))
fig.add_trace(go.Scatter(x=result_df['Date'], y=ema_list, name= 'EMA',line=dict(color="#FF7777")))
fig.update_layout(title="Closing prices and EMA",
                          xaxis_title="Date",
                          yaxis_title="Closing prices")
fig.show()